In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install -q transformers accelerate langchain_huggingface bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.6 MB/s eta 0:00:00


In [3]:
!pip -q install torch==2.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.1.0 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.1.0 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.1.0 which is incompatible.


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from langchain_huggingface import HuggingFacePipeline
from transformers import BitsAndBytesConfig, pipeline
from datetime import datetime
from functools import cache

In [5]:
@cache
def get_tokenizer_model():
  model = AutoModelForCausalLM.from_pretrained(
      "meta-llama/Llama-2-7b-chat-hf",
      device_map="auto",
      quantization_config=BitsAndBytesConfig(load_in_8bit=True),
      cache_dir="./cache"
  )
  tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir='./cache')
  return model, tokenizer

In [6]:
model, tokenizer = get_tokenizer_model()

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
def generate_output(input_text):
  inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
  streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
  output = model.generate(
      **inputs,
      streamer=streamer,
      max_new_tokens=float("inf"),
      use_cache=True
  )
  output_text = tokenizer.decode(output[0], skip_special_tokens=True)
  return output_text

## Flask API for llama-2 model



In [ ]:
!pip -q install fastapi uvicorn pyngrok colabcode nest-asyncio

In [10]:
!ngrok config add-authtoken 2IeOitDBgJ6jr7xKap3DQAzkaqs_2SAicHBCB7vJAwB6Johxr

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from fastapi import FastAPI
from pydantic import BaseModel
from colabcode import ColabCode
from pyngrok import ngrok
import nest_asyncio
import uvicorn

app = FastAPI()

class TextItem(BaseModel):
    text: str

@app.post("/post")
def create_item(item: TextItem):
    output = generate_output(item.text)
    return {"content": output}

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://a5bc-34-16-181-235.ngrok-free.app


INFO:     Started server process [413]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     45.51.26.112:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     45.51.26.112:0 - "GET /openapi.json HTTP/1.1" 200 OK


I'm doing well, thanks for asking! How about you?

It's great to hear from you, and I hope you're having a wonderful day. Is there anything new and exciting happening in your life?

I'm looking forward to catching up with you and hearing all about what's been going on. Let me know if you have any news or updates to share.

Take care and talk to you soon!
INFO:     45.51.26.112:0 - "POST /post HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [413]


KeyboardInterrupt: 